<a href="https://colab.research.google.com/github/brit228/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_141_Statistics_Probability_and_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 141
## Statistics, Probability, and Inference

## Prepare - examine what's available in SciPy

As we delve into statistics, we'll be using more libraries - in particular the [stats package from SciPy](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html).

In [0]:
from scipy import stats
dir(stats)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_binned_statistic',
 '_constants',
 '_continuous_distns',
 '_discrete_distns',
 '_distn_infrastructure',
 '_distr_params',
 '_multivariate',
 '_stats',
 '_stats_mstats_common',
 '_tukeylambda_stats',
 'absolute_import',
 'alpha',
 'anderson',
 'anderson_ksamp',
 'anglit',
 'ansari',
 'arcsine',
 'argus',
 'bartlett',
 'bayes_mvs',
 'bernoulli',
 'beta',
 'betaprime',
 'binned_statistic',
 'binned_statistic_2d',
 'binned_statistic_dd',
 'binom',
 'binom_test',
 'boltzmann',
 'boxcox',
 'boxcox_llf',
 'boxcox_normmax',
 'boxcox_normplot',
 'bradford',
 'burr',
 'burr12',
 'cauchy',
 'chi',
 'chi2',
 'chi2_contingency',
 'chisquare',
 'circmean',
 'circstd',
 'circvar',
 'combine_pvalues',
 'contingency',
 'cosine',
 'crystalball',
 'cumfreq',
 'describe',
 'dgamma',
 'dirichlet',
 'distributions',
 'division',
 'dlaplace',
 'dweibull',
 'energy_dist

In [0]:
# As usual, lots of stuff here! There's our friend, the normal distribution
norm = stats.norm()
print(norm.mean())
print(norm.std())
print(norm.var())

0.0
1.0
1.0


In [0]:
# And a new friend - t
t1 = stats.t(5)  # 5 is df "shape" parameter
print(t1.mean())
print(t1.std())
print(t1.var())

0.0
1.2909944487358056
1.6666666666666667


![T distribution PDF with different shape parameters](https://upload.wikimedia.org/wikipedia/commons/4/41/Student_t_pdf.svg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/Student's_t-distribution#/media/File:Student_t_pdf.svg))*

The t-distribution is "normal-ish" - the larger the parameter (which reflects its degrees of freedom - more input data/features will increase it), the closer to true normal.

In [0]:
t2 = stats.t(30)  # Will be closer to normal
print(t2.mean())
print(t2.std())
print(t2.var())

0.0
1.0350983390135313
1.0714285714285714


Why is it different from normal? To better reflect the tendencies of small data and situations with unknown population standard deviation. In other words, the normal distribution is still the nice pure ideal in the limit (thanks to the central limit theorem), but the t-distribution is much more useful in many real-world situations.

History sidenote - this is "Student":

![William Sealy Gosset](https://upload.wikimedia.org/wikipedia/commons/4/42/William_Sealy_Gosset.jpg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/File:William_Sealy_Gosset.jpg))*

His real name is William Sealy Gosset, and he published under the pen name "Student" because he was not an academic. He was a brewer, working at Guinness and using trial and error to determine the best ways to yield barley. He's also proof that, even 100 years ago, you don't need official credentials to do real data science!

## Live Lecture - let's perform and interpret a t-test

We'll generate our own data, so we can know and alter the "ground truth" that the t-test should find. We will learn about p-values and how to interpret "statistical significance" based on the output of a hypothesis test. We will also dig a bit deeper into how the test statistic is calculated based on the sample error, and visually what it looks like to have 1 or 2 "tailed" t-tests.

In [0]:
# TODO - during class, but please help!

## Assignment - apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [30]:
import pandas as pd
from scipy import stats

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data", header=None)
df.columns = ["Party",
              "handicapped-infants",
             "water-project-cost-sharing",
             "adoption-of-the-budget-resolution",
             "physician-fee-freeze",
             "el-salvador-aid",
             "religious-groups-in-schools",
             "anti-satellite-test-ban",
             "aid-to-nicaraguan-contras",
             "mx-missile",
             "immigration",
             "synfuels-corporation-cutback",
             "education-spending",
             "superfund-right-to-sue",
             "crime",
             "duty-free-exports",
             "export-administration-act-south-africa"]

df = df.replace({"?": None, "y": 1, "n": 0})

df.head()

,Party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [38]:
issue_dem = []
issue_rep = []
issue_all = []
issue_oth = []

for col in df:
  if col != "Party":
    rep = df[df["Party"] == "republican"][col].dropna()
    dem = df[df["Party"] == "democrat"][col].dropna()
    if stats.ttest_ind(rep, dem)[1] > 0.1:
      issue_all.append([col, stats.ttest_ind(rep, dem)[1], dem.mean(), rep.mean()])
    elif stats.ttest_ind(rep, dem)[1] < 0.01:
      if dem.mean() > rep.mean():
        issue_dem.append([col, stats.ttest_ind(rep, dem)[1], dem.mean(), rep.mean()])
      else:
        issue_rep.append([col, stats.ttest_ind(rep, dem)[1], dem.mean(), rep.mean()])
    else:
      issue_oth.append([col, stats.ttest_ind(rep, dem)[1], dem.mean(), rep.mean()])
        
print("Issues Democrats Support More Than Republicans:")
print("\tP\t\t\tDemocrat Support\tRepublican Support\tIssue")
for i in issue_dem:
  print("\t{}\t{}\t{}\t{}".format(i[1], i[2], i[3], i[0]))
print()
print()
print("Issues Republicans Support More Than Democrats:")
print("\tP\t\t\tDemocrat Support\tRepublican Support\tIssue")
for i in issue_rep:
  print("\t{}\t{}\t{}\t{}".format(i[1], i[2], i[3], i[0]))
print()
print()
print("Issues With Equal Support:")
print("\tP\t\t\tDemocrat Support\tRepublican Support\tIssue")
for i in issue_all:
  print("\t{}\t{}\t{}\t{}".format(i[1], i[2], i[3], i[0]))
print()
print()
print("Issues With Somewhat Equal Support:")
print("\tP\t\t\tDemocrat Support\tRepublican Support\tIssue")
for i in issue_oth:
  print("\t{}\t{}\t{}\t{}".format(i[1], i[2], i[3], i[0]))

Issues Democrats Support More Than Republicans:
	P			Democrat Support	Republican Support	Issue
	1.613440327936998e-18	0.6046511627906976	0.18787878787878787	handicapped-infants
	2.0703402795405602e-77	0.8884615384615384	0.13414634146341464	adoption-of-the-budget-resolution
	8.521033017443427e-31	0.7722007722007722	0.24074074074074073	anti-satellite-test-ban
	2.824718413723432e-54	0.8288973384030418	0.15286624203821655	aid-to-nicaraguan-contras
	5.030792653107883e-47	0.7580645161290323	0.11515151515151516	mx-missile
	1.5759322301054227e-15	0.5058823529411764	0.1320754716981132	synfuels-corporation-cutback
	5.997697174348817e-32	0.6374501992031872	0.08974358974358974	duty-free-exports
	3.652674361672202e-11	0.9351351351351351	0.6575342465753424	export-administration-act-south-africa


Issues Republicans Support More Than Democrats:
	P			Democrat Support	Republican Support	Issue
	1.994262314074572e-177	0.05405405405405406	0.9878787878787879	physician-fee-freeze
	5.600520111728605e-68	0.21

In [46]:
def ttest(dataframe, class_name, identity_name, val1, val2):
  out = []
  names = sorted(set(dataframe[class_name]))
  for m in range(len(names)-1):
    for n in range(m+1, len(names)):
      if m != n:
        cat1 = df[df[class_name] == names[m]][identity_name].dropna()
        cat2 = df[df[class_name] == names[n]][identity_name].dropna()
        t_test = stats.ttest_ind(cat1, cat2)[1]
        if cat1.mean() >= cat2.mean() and t_test < val1:
          out.append(("Difference", names[m], names[m], cat1.mean(), names[n], cat2.mean(), t_test))
        elif cat1.mean() < cat2.mean() and t_test <= val1:
          out.append(("Difference", names[n], names[m], cat1.mean(), names[n], cat2.mean(), t_test))
        elif t_test >= val2:
          out.append(("Similar", None, names[m], cat1.mean(), names[n], cat2.mean(), t_test))
  return out

for col in df:
  if col != "Party":
    print(col, ttest(df, "Party", col, 0.01, 0.1))

handicapped-infants [('Difference', 'democrat', 'democrat', 0.6046511627906976, 'republican', 0.18787878787878787, 1.613440327936998e-18)]
water-project-cost-sharing [('Similar', None, 'democrat', 0.502092050209205, 'republican', 0.5067567567567568, 0.9291556823994811)]
adoption-of-the-budget-resolution [('Difference', 'democrat', 'democrat', 0.8884615384615384, 'republican', 0.13414634146341464, 2.0703402795405602e-77)]
physician-fee-freeze [('Difference', 'republican', 'democrat', 0.05405405405405406, 'republican', 0.9878787878787879, 1.994262314074572e-177)]
el-salvador-aid [('Difference', 'republican', 'democrat', 0.21568627450980393, 'republican', 0.9515151515151515, 5.600520111728605e-68)]
religious-groups-in-schools [('Difference', 'republican', 'democrat', 0.47674418604651164, 'republican', 0.8975903614457831, 2.393672252059893e-20)]
anti-satellite-test-ban [('Difference', 'democrat', 'democrat', 0.7722007722007722, 'republican', 0.24074074074074073, 8.521033017443427e-31)]
aid

# Resources

- https://homepage.divms.uiowa.edu/~mbognar/applets/t.html
- https://rpsychologist.com/d3/tdist/
- https://gallery.shinyapps.io/tdist/
- https://en.wikipedia.org/wiki/Standard_deviation#Sample_standard_deviation_of_metabolic_rate_of_northern_fulmars